In [4]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices,ComplementrySubjects,SubjectGroups
from Manage.serializers import PermanentSubjectSerializer
from StakeHolders.serializers import StudentSerializer
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string
from django.db.models import Count, Q
from django.core.cache import cache

In [25]:
subject_choice = SubjectChoices.objects.filter(slug='9bd05bff4e0d43db_1731959900012').first()
finalized_subjects_set = subject_choice.finalized_choices.order_by('orderedfinalizedsubject__ordering')
for subject in finalized_subjects_set:
        # Access the related ordering object
        ordering_relation = subject.ordered_finalized_subjects.first()  # Use correct related_name
        if ordering_relation:
            ordering_value = ordering_relation.ordering
            print(f"Subject: {subject}, Ordering: {ordering_value}")
        else:
            print(f"Subject: {subject}, No ordering relation found")

AttributeError: 'Subject' object has no attribute 'ordered_finalized_subjects'

In [4]:

file_path='/home/manav1011/Documents/SMARTROLL_SSIP_2023/SMARTROLL/notebooks/sheets/electives.xlsx'

In [5]:
sheet_names = pd.ExcelFile(file_path).sheet_names
CD = pd.read_excel(file_path, sheet_name='CD')
AI = pd.read_excel(file_path, sheet_name='AI')
ML = pd.read_excel(file_path, sheet_name='ML')
BDA = pd.read_excel(file_path, sheet_name='BDA')
IS = pd.read_excel(file_path, sheet_name='IS')
MCWC = pd.read_excel(file_path, sheet_name='MCWC')
CC = pd.read_excel(file_path, sheet_name='CC')
MAD = pd.read_excel(file_path, sheet_name='MAD')
DF = pd.read_excel(file_path, sheet_name='DF')

In [6]:
semester_obj = Semester.objects.get(slug='914150_1731255589')

In [7]:
def update_student_subjects(df, subject_slug):
    subject = Subject.objects.get(slug=subject_slug)
    for _, row in df.iterrows():
        # get the student obj
        student_obj = Student.objects.get(enrollment=row['enrollment'])
        # get the subject choice 
        subject_choices_obj = SubjectChoices.objects.get(semester=semester_obj,profile=student_obj.profile)
        if subject.is_elective:
            complementries = ComplementrySubjects.objects.filter(subjects=subject).first().subjects.exclude(id=subject.id)
            for complementry in complementries:
                subject_choices_obj.available_choices.remove(complementry)
        subject_choices_obj.finalized_choices.add(subject, through_defaults={'ordering': 1})
        subject_choices_obj.available_choices.remove(subject)
        subject_choices_obj.choices_locked=True
        subject_choices_obj.save()
        

In [8]:
subjects=Subject.objects.filter(semester=semester_obj)
for subject in subjects:
    print(subject.subject_map.subject_name ," - ", subject.slug)

Compiler Design  -  0ebc225d449b4ae8_1731920560117
Mobile Computing and Wireless Communication  -  4551cd1bba6745cd_1731920560143
Cloud Computing  -  91d52b69e60149a5_1731920560174
Artificial Intelligence  -  8fb17e6790924fcd_1731920560223
Big Data Analytics  -  780ed459211a4c5d_1731920560299
Machine Learning  -  06c3643d71714da0_1731920560328
Digital forensics  -  5bd48d21484048c4_1731920560358
Mobile Application Development  -  5918c532c1e14410_1731920560378
Information security  -  a59eeda75ce3430c_1731920560261


In [10]:
update_student_subjects(ML,'06c3643d71714da0_1731920560328')
update_student_subjects(BDA,'780ed459211a4c5d_1731920560299')
update_student_subjects(IS,'a59eeda75ce3430c_1731920560261')
update_student_subjects(MCWC,'4551cd1bba6745cd_1731920560143')
update_student_subjects(CC,'91d52b69e60149a5_1731920560174')
update_student_subjects(MAD,'5918c532c1e14410_1731920560378')
update_student_subjects(DF,'5bd48d21484048c4_1731920560358')
update_student_subjects(AI,'8fb17e6790924fcd_1731920560223')
update_student_subjects(CD,'0ebc225d449b4ae8_1731920560117')

In [11]:
subject_groups = {}
student_subject_choices = SubjectChoices.objects.filter(
    semester=semester_obj, 
    profile__role='student'
).order_by('profile__student__enrollment')
for subject_choice_obj in student_subject_choices:
    student = subject_choice_obj.profile.student_set.first()
    subject_group = subject_choice_obj.finalized_choices.all()
    subject_group_tuple = tuple(subject_group)
    if subject_group_tuple not in subject_groups:
        subject_groups[subject_group_tuple] = [student]
    else:
        subject_groups[subject_group_tuple].append(student)

for subjects,students in subject_groups.items():    
    subject_groups = SubjectGroups.objects.filter(semester=semester_obj,subjects__in=subjects)
    subject_ids = [subject.id for subject in subjects]
    matching_groups = SubjectGroups.objects.filter(
        semester=semester_obj,  # Match semester
        subjects__id__in=subject_ids  # Include the specified subjects
    ).annotate(subject_count=Count('subjects')).filter(subject_count=len(subject_ids))
    if not matching_groups.exists():
        subject_group_obj = SubjectGroups.objects.create(semester=semester_obj)
        subject_group_obj.subjects.add(*subjects)
        subject_group_obj.students.add(*students)
    else:
        print(matching_groups)

<QuerySet [<SubjectGroups: Semester - 7 | Stream - BE | COMPUTER ENGINEERING | Term - 2024 | 2025>]>


In [2]:
from django.core.mail import EmailMessage
from django.template.loader import render_to_string
def send_welcome_email(user_email, user_name, temp_password, login_url):
    # Create the email subject
    subject = 'Welcome to Smartroll'

    # Prepare context for HTML email template
    context = {
        'user_name': user_name,
        'temporary_password': temp_password,
        'login_url': login_url,
    }

    # Render the HTML content using the template string
    html_message = render_to_string('temppassword.html', context)

    # Create the email message
    email = EmailMessage(
        subject=subject,
        body=html_message,
        from_email='smartroll.hq@gmail.com',  # Replace with your email
        to=[user_email],  # Recipient's email
    )    
    email.content_subtype = 'html'
    email.send()
    print("Email sent successfully.")

In [3]:
send_welcome_email('manavshah1011.ms@gmail.com','Manav Shah','skmakdga','smartroll@mnv-dev.live')
send_welcome_email('parmarharsh901@gmail.com','Harsh Parmar','skmakdga','smartroll@mnv-dev.live')
send_welcome_email('amansevak123@gmail.com','Aman Bhatt','skmakdga','smartroll@mnv-dev.live')
send_welcome_email('pandyaprathmesh360@gmail.com','Prathmesh Pandya','skmakdga','smartroll@mnv-dev.live')
send_welcome_email('dhrumilpanchal1434@gmail.com','Dhrumil Panchal','skmakdga','smartroll@mnv-dev.live')

Email sent successfully.
Email sent successfully.
Email sent successfully.
Email sent successfully.
Email sent successfully.
